# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [4]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("London")

Tool get_ticket_price called for London


'$799'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
# And this is included in a list of tools:

#tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [10]:
# This use case is useful, because its a REAl TIME API call.

import requests
import random


def get_price_from_api(city):
    print(f">>>>> Tool [get_price_from_api] called for \"{city}\"")

    try:
        product_id = str(random.randint(1, 189))
        url = 'https://dummyjson.com/products/' + product_id
        response = requests.get(url)
        print(f"Called {url} ---> {response}")
        response.raise_for_status()  # Raises an exception for bad status codes
        data = response.json()
#        print(json.dumps(data, indent=4))
        return data['price']
        
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")# There's a particular dictionary structure that's required to describe our function:

price_api_function = {
    "name": "get_price_from_api",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [11]:
# We have to write that function handle_tool_call:

def handle_tool_call(message_from_model):
    tool_call = message_from_model.tool_calls[0]
    print(f"tool_call is ======> {tool_call}")

    # Here we could test for different tool_calls functons:
    if tool_call.function.name == 'get_price_from_api':        
        print(f"Executing tool_call for [get_price_from_api]...")
    
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
    
        print(f"Received args from Model: {json.dumps(arguments, indent=2)}")
        print("*** ===================================================")

        # TOOL: Call to ext API!  (This is the juicy part!)
        price = get_price_from_api(city)

        # Pack response to Model (with result and id):
        response = {
            "role": "tool", # New role! system, user, assistant, tool
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id # id required for messages with role 'tool'
        }
        return response, None

    if tool_call.function.name == 'artist_tool':        
     
        print(f"Executing tool_call for [get_price_from_api]...")
    
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('city_name')
    
        print(f"Received args from Model: {json.dumps(arguments, indent=2)}")
        print("*** ===================================================")

        # Call to ext API!  (This is the juicy part!)
        img = artist(city)

        # Pack response to Model (with result and id):

#        return response, city
        return None, img        

In [12]:
#gr.ChatInterface(fn=chat, type="messages").launch()

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [13]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [14]:
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
  #          model="dall-e-2",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
  #          size="512x512",
            quality="standard",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [15]:
#image = artist("New York City")
#display(image)

In [16]:
# There's a particular dictionary structure that's required to describe our function:

artist_tool_function = {
    "name": "artist_tool",
    "description": "Get an image of the destination city. Call this whenever you need to show an image of the destination city, or when a customer asks any visual related question about the destination city, for example 'What does this city look like' or 'Is this a pretty city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city_name": {
                "type": "string",
                "description": "The name of the city that the customer wants to travel to",
            },
        },
        "required": ["city_name"],
        "additionalProperties": False
    }
}

In [17]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_api_function}]
tools.append({"type": "function", "function": artist_tool_function})
print(tools)

[{'type': 'function', 'function': {'name': 'get_price_from_api', 'description': "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'", 'parameters': {'type': 'object', 'properties': {'destination_city': {'type': 'string', 'description': 'The city that the customer wants to travel to'}}, 'required': ['destination_city'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'name': 'artist_tool', 'description': "Get an image of the destination city. Call this whenever you need to show an image of the destination city, or when a customer asks any visual related question about the destination city, for example 'What does this city look like' or 'Is this a pretty city'", 'parameters': {'type': 'object', 'properties': {'city_name': {'type': 'string', 'description': 'The name of the city that the customer wants to travel to'}}, 'required': ['city_name

In [18]:
def chat_w_images(message, history):
#def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    print(f"\n\n\nCURRENT MESSAGE STACK >>: \n\n")
    for msg in messages:
        print(f"\n{msg}\n")
    print(f"\n ---- END (current stack) ----\n")

    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    finish_reason = response.choices[0].finish_reason
    print(f"The response finished because: {finish_reason}")

#    if response.choices[0].finish_reason=="tool_calls":
    if finish_reason=="tool_calls":
        message_from_model = response.choices[0].message
        tool_call_id = message_from_model.tool_calls[0].id
        print("*** The Model wants us to make an external API call  ==")
        print(f"*** TOOL CALL with ID = {tool_call_id}              ==")
#        response, city = handle_tool_call(message_from_model)
#        print(f"The response, city: {response,city}")
        response, img = handle_tool_call(message_from_model)
        print(f"The response: {response}")

        # IMPORTANT - here we return the request & response to the model so it can access the result.
        # Each has the tool_call_id embedded so the Model knows to connect them.
        messages.append(message_from_model)
        messages.append(response)

        print(f"\n\n\nTOOL CALLS MESSAGE >>: \n\n")
        for msg in messages:
            print(f"\n{msg}\n")
        print("---- END (tool_calls stack) ----")
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content, img

In [19]:
#gr.ChatInterface(fn=chat_w_images, type="messages").launch(inbrowser=True)

In [20]:
# I deleted ALL the audio cells.

# Our Agent Framework

The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The 'Agent Environment' which allows Agents to collaborate
4. An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

We're showing 1 and 2 here, and to a lesser extent 3 and 5. In week 8 we will do the lot!

In [21]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, img = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        #image = artist(city)
        image = img
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    #talker(reply)
    
    return history, image

In [22]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
  #      chat_w_images, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2044, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1591, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return

# Exercises and Business Applications

Add in more tools - perhaps to simulate actually booking a flight. A student has done this and provided their example in the community contributions folder.

Next: take this and apply it to your business. Make a multi-modal AI assistant with tools that could carry out an activity for your work. A customer support assistant? New employee onboarding assistant? So many possibilities! Also, see the week2 end of week Exercise in the separate Notebook.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a HUGE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>